## Imports and settings

In [1]:
import src
import keras.backend as K
import os
import numpy as np
import sys
import re
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from  matplotlib.animation import FuncAnimation
from matplotlib import colors
from netCDF4 import Dataset
from IPython.display import clear_output
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')
#forces CPU usage
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" #"" or "-1" for CPU, "0" for GPU
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2447111106651953373
]


______________________________________-
## Load datasets

In [3]:
train, xval, test = src.load_datasets("gan")

Training data: (6000, 4, 64, 64, 3)
Validation data: (2000, 4, 64, 64, 3)
Test data: (2000, 4, 64, 64, 3)


In [ ]:
%matplotlib notebook
#visualise channels
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4,figsize=(9,9))
for ax in [ax1,ax2,ax3,ax4]:
    ax.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    left=False,      # ticks along the top edge are off
    labelbottom=False,
    labelleft=False) # labels along the bottom edge are off
line1 = ax1.imshow(train[0,0,:,:,0]) # start from index 0 which means 0:50:00
line2 = ax2.imshow(train[0,1,:,:,0]) # start from index 1 which means 1:00:00
line3 = ax3.imshow(train[0,2,:,:,0])
line4 = ax4.imshow(train[0,3,:,:,0])

line = [line1, line2, line3, line4]

def run(i):
    line[0].set_data(train[i,0,:,:,0])
    line[1].set_data(train[i,1,:,:,0])
    line[2].set_data(train[i,2,:,:,0])
    line[3].set_data(train[i,3,:,:,0])
    fig.suptitle(f"Index in array: {i}")
    return line

ani = FuncAnimation(fig, run, blit=True, interval=200, frames=100,
    repeat=False)
plt.show()

In [ ]:
src.visualise_data(train)

## Generator

U-net

In [ ]:
#model2=keras.Sequential()
def unet():
    init       = keras.layers.Input(shape=(64,64,1))
    ConvDown1  = keras.layers.Conv2D(filters=8,kernel_size=(2,2),strides=(1,1),padding="same")(init)
    Lr1        = keras.layers.LeakyReLU(alpha=0.1)(ConvDown1)
    #64
    ConvDown2  = keras.layers.Conv2D(filters=16,kernel_size=(2,2),strides=(2,2),padding="same")(Lr1)
    Lr2        = keras.layers.LeakyReLU(alpha=0.1)(ConvDown2)
    #32
    ConvDown3  = keras.layers.Conv2D(filters=32,kernel_size=(2,2),strides=(2,2),padding="same")(Lr2)
    Lr3        = keras.layers.LeakyReLU(alpha=0.1)(ConvDown3)
    #16
    ConvDown4  = keras.layers.Conv2D(filters=32,kernel_size=(2,2),strides=(2,2),padding="same")(Lr3)
    Lr4        = keras.layers.LeakyReLU(alpha=0.1)(ConvDown4)
    #8
    ConvDown5  = keras.layers.Conv2D(filters=32,kernel_size=(2,2),strides=(2,2),padding="same")(Lr4)
    Lr5        = keras.layers.LeakyReLU(alpha=0.1)(ConvDown5)
    #4

    UpSamp1 = keras.layers.UpSampling2D(size=(2, 2), data_format="channels_last")(Lr5)
    #8
    merge1  = keras.layers.concatenate([ConvDown4,UpSamp1],axis=-1)#(UpSamp1)
    Conv1   = keras.layers.Conv2D(filters=32,kernel_size=(4,4),strides=(1,1),padding="same")(merge1)
    Lr6     = keras.layers.LeakyReLU(alpha=0.1)(Conv1)
    #8
    UpSamp2 = keras.layers.UpSampling2D(size=(2, 2), data_format="channels_last")(Lr6)
    #16
    merge2  = keras.layers.concatenate([ConvDown3,UpSamp2],axis=-1)#(UpSamp2)
    Conv2   = keras.layers.Conv2D(filters=32,kernel_size=(4,4),strides=(1,1),padding="same")(merge2)
    Lr7     = keras.layers.LeakyReLU(alpha=0.1)(Conv2)
    #16
    UpSamp3 = keras.layers.UpSampling2D(size=(2, 2), data_format="channels_last")(Lr7)

    #32
    Conv3   = keras.layers.Conv2D(filters=16,kernel_size=(4,4),strides=(1,1),padding="same")(UpSamp3)
    Lr8     = keras.layers.LeakyReLU(alpha=0.1)(Conv3)

    UpSamp4 = keras.layers.UpSampling2D(size=(2, 2), data_format="channels_last")(Lr8)
    #64
    Conv4   = keras.layers.Conv2D(filters=8,kernel_size=(4,4),strides=(1,1),padding="same")(UpSamp4)
    Lr9     = keras.layers.LeakyReLU(alpha=0.1)(Conv4)
    
    Conv5   = keras.layers.Conv2D(filters=1,kernel_size=(4,4),strides=(1,1),padding="same",activation = 'tanh')(Lr9)
    
    return keras.models.Model(inputs=init, outputs=Conv5)

## Discriminator

In [ ]:
def spatial_discriminator():
    dropout = 0.5
    init = keras.layers.Input(shape=(64,64,1))
    
    conv1 = keras.layers.Conv2D(filters=4 ,kernel_size=4, strides=2, padding='same')(init)
    relu1 = keras.layers.LeakyReLU(alpha=0.2)(conv1)
    dropout1 = keras.layers.Dropout(dropout)(relu1)
    
    conv2 = keras.layers.Conv2D(filters=8, kernel_size=4, strides=2, padding='same')(dropout1)
    relu2 = keras.layers.LeakyReLU(alpha=0.2)(conv2)
    dropout2 = keras.layers.Dropout(dropout)(relu2)
    
    conv3 = keras.layers.Conv2D(filters=16, kernel_size=4, strides=2, padding='same')(dropout2)
    relu3 = keras.layers.LeakyReLU(alpha=0.2)(conv3)
    dropout3 = keras.layers.Dropout(dropout)(relu3)
    
    conv4 = keras.layers.Conv2D(filters=32, kernel_size=4, strides=2, padding='same')(dropout3)
    relu4 = keras.layers.LeakyReLU(alpha=0.2)(conv4)
    dropout4 = keras.layers.Dropout(dropout)(relu4)
    
    # Out: 1-dim probability
    flatten = keras.layers.Flatten()(dropout4)
    fcl1 = keras.layers.Dense(1)(flatten)
    sig1 = keras.layers.Activation('sigmoid')(fcl1)
    
    return keras.models.Model(inputs=init, outputs=sig1)

In [ ]:
def temporal_discriminator():
    dropout = 0.5
    init = keras.layers.Input(shape=(2,64,64,3)) # t - 1, t
    
    conv1 = keras.layers.Conv2D(filters=4 ,kernel_size=4, strides=2, padding='same')(init)
    relu1 = keras.layers.LeakyReLU(alpha=0.2)(conv1)
    dropout1 = keras.layers.Dropout(dropout)(relu1)
    
    conv2 = keras.layers.Conv2D(filters=8, kernel_size=4, strides=2, padding='same')(dropout1)
    relu2 = keras.layers.LeakyReLU(alpha=0.2)(conv2)
    dropout2 = keras.layers.Dropout(dropout)(relu2)
    
    conv3 = keras.layers.Conv2D(filters=16, kernel_size=4, strides=2, padding='same')(dropout2)
    relu3 = keras.layers.LeakyReLU(alpha=0.2)(conv3)
    dropout3 = keras.layers.Dropout(dropout)(relu3)
    
    conv4 = keras.layers.Conv2D(filters=32, kernel_size=4, strides=2, padding='same')(dropout3)
    relu4 = keras.layers.LeakyReLU(alpha=0.2)(conv4)
    dropout4 = keras.layers.Dropout(dropout)(relu4)
    
    # Out: 1-dim probability
    flatten = keras.layers.Flatten()(dropout4)
    fcl1 = keras.layers.Dense(1)(flatten)
    sig1 = keras.layers.Activation('sigmoid')(fcl1)
    
    return keras.models.Model(inputs=init, outputs=sig1)

In [ ]:
def advect(image): # (64,64,3)
    padded = np.pad(image,(0,1),'edge')[:,:,:-1]
    advected = np.empty_like(image)
    advected[:,:,0] = image[:,:,0] + image[:,:,1]*(padded[1:,:,0] - padded[:-1,:,0])[:,:-1] + image[:,:,2]*(padded[:,1:,0] - padded[:,:-1,0])[:-1]
    advected[:,:,1:] = image[:,:,1:]
    return advected

In [21]:
a,b,c,d,e,f = split_datasets(train, xval, test, past_frames=1, future_frames=2)

Shape of training data:  (6000, 64, 64, 3) 
Shape of training truth:  (6000, 2, 64, 64, 3) 
Shape of validation data:  (2000, 64, 64, 3) 
Shape of validation truth:  (2000, 2, 64, 64, 3) 
Shape of test data:  (2000, 64, 64, 3) 
Shape of test truth:  (2000, 2, 64, 64, 3)


In [11]:
def split_datasets(train, xval, test, past_frames=1, future_frames=1):
    assert past_frames + future_frames <= train.shape[1], "Wrong frame specification!"
    assert past_frames > 0, "No. of past frames must be a positive integer!"
    assert future_frames > 0,  "No. of future frames must be a positive integer!"
    training_data         = np.reshape(train[:,:past_frames,:,:,:],
                                       ((train.shape[0],)+train.shape[2:]) if past_frames == 1 else ((train.shape[0],past_frames)+train.shape[2:]))
    trainig_data_truth    = np.reshape(train[:,past_frames:past_frames+future_frames,:,:,:],
                                       ((train.shape[0],)+train.shape[2:]) if future_frames == 1 else ((train.shape[0],future_frames)+train.shape[2:]))
    validation_data       = np.reshape(xval[:,:past_frames,:,:,:],
                                       ((xval.shape[0],)+xval.shape[2:]) if past_frames == 1 else ((xval.shape[0],past_frames)+xval.shape[2:]))
    validation_data_truth = np.reshape(xval[:,past_frames:past_frames+future_frames,:,:,:],
                                       ((xval.shape[0],)+xval.shape[2:]) if future_frames == 1 else ((xval.shape[0],future_frames)+xval.shape[2:]))
    test_data             = np.reshape(test[:,:past_frames,:,:,:],
                                       ((test.shape[0],)+test.shape[2:]) if past_frames == 1 else ((test.shape[0],past_frames)+test.shape[2:]))
    test_data_truth       = np.reshape(test[:,past_frames:past_frames+future_frames,:,:,:],
                                       ((test.shape[0],)+test.shape[2:]) if future_frames == 1 else ((test.shape[0],future_frames)+test.shape[2:]))
    
    print("Shape of training data: ",training_data.shape,"\nShape of training truth: ",trainig_data_truth.shape,
    "\nShape of validation data: ",validation_data.shape,"\nShape of validation truth: ",validation_data_truth.shape,
    "\nShape of test data: ",test_data.shape,"\nShape of test truth: ",test_data_truth.shape)
    return training_data, trainig_data_truth, validation_data, validation_data_truth, test_data, test_data_truth

In [ ]:
#modified from source: https://github.com/eriklindernoren/Keras-GAN/blob/master/gan/gan.py
class GAN():
    def __init__(self):
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.metric = [src.relative_error_tensor]

        d_optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        g_optimizer = keras.optimizers.Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=d_optimizer,
            metrics=["accuracy"])
        # Build the generator
        self.generator = self.build_generator()
        #self.generator.compile(loss='mse',
        #    optimizer=g_optimizer,
        #    metrics=self.metric)
        
        # The generator takes a sequence of frames as input and generates the next image in that sequence
        input_img = keras.layers.Input(shape=self.img_shape)
        generated = self.generator(input_img)
        
        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.discriminator.trainable = False
        self.combined = keras.models.Model(input_img, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=g_optimizer)


    def build_generator(self,network="U-net"):  
        generator = keras.Sequential()
        if network in ["Unet", "U-net", "unet", "u-net"]:
            return unet()

    def build_discriminator(self, which="s"):
        if which == "s":
            return spatial_discriminator()
        elif which == "t":
            return temporal_discriminator()

    def train(self, epochs, g_epochs=1, d_epochs=1, dataset="5min", batch_size=128, dual=False):

        # Load the dataset
        train, xval, test = src.load_datasets(dataset=dataset)
        
        # split the dataset
        gan_train     = np.reshape(train[:,0,:,:,:],((train.shape[0],)+train.shape[2:]))
        gan_truth     = np.reshape(train[:,1,:,:,:],((train.shape[0],)+train.shape[2:]))
        gan_val       = np.reshape(xval[:,0,:,:,:],((xval.shape[0],)+xval.shape[2:]))
        gan_val_truth = np.reshape(xval[:,1,:,:,:],((xval.shape[0],)+xval.shape[2:]))
        gan_test      = np.reshape(test[:,0,:,:,:],((test.shape[0],)+test.shape[2:]))
        gan_test_truth = np.reshape(test[:,1,:,:,:],((test.shape[0],)+test.shape[2:]))
        
        print("Shape of training data: ",gan_train.shape,"\nShape of training truth: ",gan_truth.shape,
        "\nShape of validation data: ",gan_val.shape,"\nShape of validation truth: ",gan_val_truth.shape,
        "\nShape of test data: ",gan_test.shape,"\nShape of test truth: ",gan_test_truth.shape)
        
        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        #store losses

        log = {"g_loss":[],
               "d_loss":[],
               "g_metric":[],
               "d_metric":[]}
        
        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminators
            # ---------------------

            idx = np.random.randint(0, gan_truth.shape[0], batch_size)
            real_imgs = gan_truth[idx]
            training_batch = gan_train[idx]

            # Generate a batch of new images
            generated_imgs = self.generator.predict(training_batch)

            # Train the first discriminator
            for ks in range(d_epochs):
                d_loss_real = self.discriminator.train_on_batch(real_imgs, real)
                d_loss_fake = self.discriminator.train_on_batch(generated_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                print(f"    {ks} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}]")
                
            
            # Train the second discriminator
            if dual:
                for kt in range(d_epochs):
                    advected = np.array([advect(sample) for sample in real_imgs])
                    d_loss_real = self.discriminator.train_on_batch(real_imgs, real)
                    d_loss_fake = self.discriminator.train_on_batch(generated_imgs, fake)
                    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                    print(f"    {ks} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}]")
            
            # ---------------------
            #  Train Generator
            # ---------------------

            idx2 = np.random.randint(0, gan_train.shape[0], batch_size)
            training_batch = gan_train[idx2]
            training_truth = gan_truth[idx2]

            # Train the generator (to have the discriminator label samples as real)
            for kg in range(g_epochs):
                g_loss = self.combined.train_on_batch(training_batch, real)

            # Plot the progress
            log["g_loss"].append(g_loss)
            log["d_loss"].append(d_loss[0])
            #log["g_metric"].append(g_loss[1])
            log["d_metric"].append(d_loss[1])
            print(f"\033[1m {epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}]\033[0m"+
                  f"\033[1m[G loss: {g_loss}]\033[0m")#, rel. err.: {g_loss[1]}] \033[0m")

            # If at save interval => save generated image samples
            if epoch in [int(x) for x in np.linspace(0.1,1,10)*epochs]:
                self.sample_images(epoch, gan_test, gan_test_truth)
        return log

    def sample_images(self, epoch, gan_test, gan_test_truth):
        n = 5
        test_batch = gan_test[:n]
        test_truth = gan_test_truth[:n]
        gen_imgs = self.generator.predict(test_batch)

        fig, axs = plt.subplots(n, 3, figsize=(16, 16))
        for i in range(n):
                axs[i,0].imshow(test_batch[i, :,:,0])
                axs[i,0].axis('off')
                axs[i,0].set_title("Frame t")
                axs[i,1].imshow(test_truth[i, :,:,0])
                axs[i,1].axis('off')
                axs[i,1].set_title("Frame t+1")
                axs[i,2].imshow(gen_imgs[i, :,:,0])
                axs[i,2].axis('off')
                axs[i,2].set_title("Prediction t+1")
        fig.savefig("Plots/epoch %d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    log = gan.train(epochs=10, d_epochs=5, batch_size=64)

In [ ]:

f = plt.figure(figsize=(16,4))
ax = f.add_subplot(121)
ax2 = f.add_subplot(122)
ax.plot(log["g_loss"],label="Generator loss")
ax.plot(log["d_loss"],label="Discriminator loss")
ax.grid()
ax.legend(loc="best")

ax2.plot(log["g_metric"],label="Generator metric")
ax2.plot(log["d_metric"],label="Discriminator metric")
ax2.grid()
ax2.legend(loc="center right")